<a href="https://colab.research.google.com/github/antussa2016/thesis_work/blob/master/data_analysis/200Tweet_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import nltk
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/tweet_files.csv')
folder_path = '/content/drive/MyDrive/tweet1/tweet'

In [ ]:
file_names = os.listdir(folder_path)

In [ ]:
csv_files = [file_name for file_name in file_names if file_name.endswith('.csv')]

# Removing similar words from tweets

In [ ]:
def removing_similar_words(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    unique_tokens = list(set(lemmatized_tokens))
    clean_tweet = ' '.join(unique_tokens)
    return clean_tweet

# **Removing preposition from tweets**

In [ ]:
def remove_prepositions(tweet):
    # Tokenize tweet into words
    words = nltk.word_tokenize(tweet)

    # Remove prepositions using list comprehension
    filtered_words = [word for word in words if word.lower() not in nltk.corpus.stopwords.words('english') and nltk.pos_tag([word])[0][1] != 'IN']

    # Rejoin filtered words into a string
    filtered_tweet = ' '.join(filtered_words)

    # Return filtered tweet
    return filtered_tweet

# **Removing unwanted words like articles from tweets**

In [ ]:
unwanted_words = ['a', 'an', 'the', 'and', 'or', 'but']

# Define a function to remove unwanted words
def remove_unwanted_words(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove unwanted words using regular expressions
    text = re.sub(r'\b(' + '|'.join(unwanted_words) + r')\b', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Example tweet

# Remove unwanted words from the tweet


# **Removing one or two letters words from tweets**

In [ ]:
def removing_oneOrtwo_words(text):
    stop_words = nltk.corpus.stopwords.words('english')
    stop_words.extend(['rt', 'co', 'u', 'ur', 'im'])
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if not word.lower() in stop_words and len(word) > 2]
    filtered_tweet = ' '.join(filtered_tokens)
    return filtered_tweet

# **Removing repeated words from tweets**

In [ ]:
def removing_repeated_words(text):
    words = text.split()
    word_counts = Counter(words)
    most_common = [word for word, count in word_counts.most_common(5)]
    words = [word for word in words if word not in most_common]
    clean_tweet = " ".join(words)
    return clean_tweet

In [ ]:
k = 0
new_column = []
for index, row in df1.iterrows():
    for file_name in csv_files:
        if row['File name'] == file_name:
            df3=file_names[k]
            df3=pd.read_csv(f'/content/drive/MyDrive/tweet1/tweet/{df3}')
            # column=df3['tweet'].name

            if ('tweet' in df3.columns) & ('text' not in df3.columns):
                df3['tweet'] =  df3['tweet'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
                df3['tweet'] = df3['tweet'].str.replace(r"[\"\'\|\?\=\.\@\#\*\,]", '')
                processed_texts = [text.lower().split() for text in df3['tweet']]
                bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
                bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
                def get_sentence_embeding(sentences):
                    preprocessed_text = bert_preprocess(sentences)
                    return bert_encoder(preprocessed_text)['pooled_output']
                text = []
                for i in df3['tweet']:
                    string = ""
                    for j in i:
                        string += j
                        text.append(string)
                bert_text = text[:200]
                bert_string = ' '.join(bert_text)
                string1 = removing_similar_words(bert_string)
                string2 = remove_prepositions(string1)
                string3 = remove_unwanted_words(string2)
                string4 = removing_oneOrtwo_words(string3)
                string5 = removing_repeated_words(string4)
                bert_list = []
                bert_list.append(string5)
                bert_embed = get_sentence_embeding(bert_list)
                hidden = tf.constant(bert_embed)
                hidden_array = hidden.numpy()
                hidden_array.resize((768,))
                hidden_values = hidden_array[:768]
                new_column.append(hidden_values)
                print(k)
                k +=1
            else:
                df3.loc[df3['text'].isna()]
                df3['text'] =  df3['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
                df3['text'] = df3['text'].str.replace(r"[\"\'\|\?\=\.\@\#\*\,]", '')
                processed_texts = [text.lower().split() for text in df3['text'].dropna().tolist()]
                bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
                bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
                def get_sentence_embeding(sentences):
                    preprocessed_text = bert_preprocess(sentences)
                    return bert_encoder(preprocessed_text)['pooled_output']
                text = []
                for i in df3['text'].dropna().tolist():
                    string = ""
                    for j in i:
                        string += j
                        text.append(string)
                bert_text = text[:200]
                bert_string = ' '.join(bert_text)
                string1 = removing_similar_words(bert_string)
                string2 = remove_prepositions(string1)
                string3 = remove_unwanted_words(string2)
                string4 = removing_oneOrtwo_words(string3)
                string5 = removing_repeated_words(string4)
                bert_list = []
                bert_list.append(string5)
                bert_embed = get_sentence_embeding(bert_list)
                hidden = tf.constant(bert_embed)
                hidden_array = hidden.numpy()
                hidden_array.resize((768,))
                hidden_values = hidden_array[:768]
                new_column.append(hidden_values)
                print(k)
                k +=1


In [ ]:
df1['new_column'] = new_column

In [ ]:
df1.head()

,File name,new_column
0,bozhobg_.csv,"[-0.83739275, -0.53459024, -0.8605162, 0.72336..."
1,engineer_fouad_.csv,"[-0.8078663, -0.34387338, -0.6524926, 0.640008..."
2,yuvadm_.csv,"[-0.8478003, -0.4863819, -0.8600512, 0.7470851..."
3,@borland_.csv,"[-0.73726755, -0.39729238, -0.5068549, 0.33198..."
4,@agriffini_.csv,"[-0.700285, -0.4379541, -0.65491354, 0.4998842..."


In [ ]:
df1.to_csv('/content/drive/MyDrive/encoded_files_new_data.csv', index=False)